In [1]:
%env http_proxy=http://127.0.0.1:10890
%env https_proxy=http://127.0.0.1:10890
%env no_proxy=127.0.0.1,localhost
%env HTTP_PROXY=http://127.0.0.1:10890
%env HTTPS_PROXY=http://127.0.0.1:10890
%env NO_PROXY=127.0.0.1,localhost
!echo -e "\033[32m[√] 已开启代理\033[0m"

env: http_proxy=http://127.0.0.1:10890
env: https_proxy=http://127.0.0.1:10890
env: no_proxy=127.0.0.1,localhost
env: HTTP_PROXY=http://127.0.0.1:10890
env: HTTPS_PROXY=http://127.0.0.1:10890
env: NO_PROXY=127.0.0.1,localhost
[√] 已开启代理


In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
from search import search_github
from download import load_readme, clone_github_repo
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

embeddings = OpenAIEmbeddings()

In [3]:
from autogen import ConversableAgent

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [4]:
import os
from autogen import ConversableAgent, register_function
from typing import Annotated, Literal
from typing import List, Dict, Tuple

from assistant import check_local
from search import search_github
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain_community.vectorstores import FAISS
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_openai import OpenAIEmbeddings
from search import search_github
from download import load_readme, clone_github_repo
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

def download_readme_to_db(keywords: List[str]) -> str:
    keywords = [keyword.lower().strip() for keyword in keywords][:1]
    db_path = check_local(keywords)
    # vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    if not db_path:
        repos = search_github(keywords, 1)
        vector_store = FAISS(
                        embedding_function=embeddings,
                        index=faiss.IndexFlatL2(len(embeddings.embed_query("hello world"))),
                        docstore=InMemoryDocstore(),
                        index_to_docstore_id={},
                    )
        load_readme(repos, vector_store)
        db_file_name = '-'.join(keywords).lower()
        db_path = f'./db/{db_file_name}'
        vector_store.save_local(db_path)
    return db_path

def search_db(db_path: str, text: str) -> List[str]:
    vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    bm25_retriever = BM25Retriever.from_documents(vector_store.docstore._dict.values())
    faiss_retriever = vector_store.as_retriever()
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, faiss_retriever], weights=[0.8, 0.2]
    )
    similar_repos = ensemble_retriever.invoke(text)
    return [{"repo_name": repo.metadata["repo_name"], "repo_readme": repo.page_content} for repo in similar_repos]

In [5]:
from prompts import Planner_prompt, Readme_Writer_prompt, Keywords_Extracter_prompt, Readme_Reader_prompt
Planner = ConversableAgent(
    name="Planner",
    system_message=Planner_prompt
)
Readme_Writer = ConversableAgent(
    name="Readme_Writer",
    system_message=Readme_Writer_prompt
)
Keyword_Extracter = ConversableAgent(
    name="Keyword_Extracter",
    system_message=Keywords_Extracter_prompt
)
Readme_Reader = ConversableAgent(
    name="Readme_Reader",
    system_message=Readme_Reader_prompt
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)
from autogen import GroupChat, GroupChatManager
group_chat = GroupChat(
    agents=[user_proxy, Planner, Readme_Writer, Keyword_Extracter, Readme_Reader],
    messages=[],
    max_round=10,
)
group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

In [6]:

register_function(
    download_readme_to_db,
    caller=group_chat_manager,
    executor=user_proxy,
    description="Search github repos with a list of detailed keywords, and save the readme of them into a vector database"
)

register_function(
    search_db,
    caller=group_chat_manager,
    executor=user_proxy,
    description="Search with the text from a vector db loaded from db_path, and return the descriptions and readme of top related repos"
)

In [10]:
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="Gaussian Splatting with C kernel on the CPU",
)

User (to chat_manager):

Gaussian Splatting with C kernel on the CPU

--------------------------------------------------------------------------------

Next speaker: Keyword_Extracter


>>>>>>>> USING AUTO REPLY...
Keyword_Extracter (to chat_manager):



--------------------------------------------------------------------------------


NameError: name 'cerebras_AuthenticationError' is not defined